In [7]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
end = datetime.date.today()
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = data.astype("double")

initial = 1000_000

In [8]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = "6M"))
dfs = [group for _, group in grouped_data]

In [9]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
sys.path.append(os.pardir)
from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

returns = []

for monthly_data in dfs:
    # バックテストを設定
    bt = Backtest(
        monthly_data, # チャートデータ
        btst.RsiOscillator_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns.append(output)

In [15]:
import statistics

df = pd.DataFrame(returns)

#TODO:最低と最大の期間のBacktestingを表示＆実行
lines = "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)
lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(df[df["# Trades"] != 0]["Return [%]"].min()) + "\n"
lines += "最大Return期間: "
lines += "最低Return [%]: " + str(df[df["# Trades"] != 0]["Return [%]"].min()) + "\n"
lines += "最低Return期間: "

print(lines)

テスト期間: 11
平均Return [%]: -4.3387
最大Return [%]: -37.8924458
最低Return [%]: -37.8924458



In [11]:
#取引ない月込み
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()